In [1]:
import os
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from keras.callbacks import TensorBoard


In [47]:
# Función para procesar los archivos MIDI
def get_notes():
    notes = []
    for file in os.listdir('songs'):
        if file.endswith(".mid"):
            midi = converter.parse(os.path.join('songs', file))
            notes_to_parse = None
            try:
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse()
            except:
                notes_to_parse = midi.flat.notes
            
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
    
    return notes


## Preprocessing

Se obtienen las notas de los archivos midi para obtener el vocabulario, el output e input del network

In [48]:
# Preprocesamiento
notes = get_notes()
n_vocab = len(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(set(notes)))

sequence_length = 100
network_input = []
network_output = []

for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)


## Creación del modelo

Se le agregan las capas al modelo

In [55]:
# Creación del modelo
model = Sequential()
model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


## Initial Loss

In [52]:
print("Evaluating initial model performance...")
initial_loss = model.evaluate(network_input, network_output, verbose=1)
print(f"Initial loss: {initial_loss}")


Evaluating initial model performance...
232/232 ━━━━━━━━━━━━━━━━━━━━ 52s 222ms/step - loss: 5.1588
Initial loss: 5.159389972686768


In [53]:
tensorboard = TensorBoard(log_dir='logs', histogram_freq=1, write_graph=True, write_images=True)

## Entrenar el modelo

In [65]:
# Entrenamiento del modelo
history = model.fit(network_input, network_output, epochs=7, batch_size=64, verbose=1)


Epoch 1/7
116/116 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - loss: 4.3696
Epoch 2/7
116/116 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - loss: 4.2016
Epoch 3/7
116/116 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - loss: 4.1696
Epoch 4/7
116/116 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - loss: 4.0569
Epoch 5/7
116/116 ━━━━━━━━━━━━━━━━━━━━ 185s 2s/step - loss: 3.9104
Epoch 6/7
116/116 ━━━━━━━━━━━━━━━━━━━━ 240s 2s/step - loss: 3.6902
Epoch 7/7
116/116 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - loss: 3.4439


## Rendimiento del modelo 

Métrica: Loss 

7 epochs - Model improvement: 38.75%


In [66]:
print("Evaluating final model performance...")
final_loss = model.evaluate(network_input, network_output, verbose=1)
print(f"Final loss: {final_loss}")

Evaluating final model performance...
232/232 ━━━━━━━━━━━━━━━━━━━━ 83s 360ms/step - loss: 2.5874
Final loss: 3.1603667736053467


In [67]:
improvement = ((initial_loss - final_loss) / initial_loss) * 100
print(f"Model improvement: {improvement:.2f}%")

Model improvement: 38.75%


## Predicciones

In [79]:
# Generar música
def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input[start]
    prediction_output = []

    for note_index in range(300):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output


In [80]:
# Nueva función para crear el archivo MIDI
def create_midi(prediction_output):
    offset = 0
    output_notes = []

    # Create the output directory if it doesn't exist
    output_dir = './gen'
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Output directory created/verified at: {output_dir}")
    except Exception as e:
        print(f"Error creating directory: {e}")
        # If we can't create the directory, save in current directory
        output_dir = '.'

    # Crear objetos note y chord basados en los valores generados
    for pattern in prediction_output:
        # Si el patrón es un acorde
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.ElectricPiano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # Si el patrón es una nota
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.ElectricPiano()
            output_notes.append(new_note)

        # Incrementar el offset para que las notas no se superpongan
        offset += 0.5

    # Crear un stream de música21
    midi_stream = stream.Stream(output_notes)

    # Escribir el stream en un archivo MIDI
    midi_stream.write('midi', fp=f'{output_dir}\\generated_song_7epochs1.mid')

    midi_stream.show('midi')

    return midi_stream


In [81]:

# Código para generar y guardar nueva música
# Asumimos que ya has entrenado el modelo y tienes los datos necesarios
prediction_output = generate_notes(model, network_input, set(notes), n_vocab)
prediction_midi = create_midi(prediction_output)

print("Archivo MIDI generado")


Output directory created/verified at: ./gen


SubConverterException: Most issues with show() can be resolved by calling configure.run()

In [77]:
type(prediction_midi)

NoneType

## Guardar modelos

In [3]:
# Save the final model
model_save_dir = "model"
final_model_path = os.path.join(model_save_dir, "final_model.h5")

In [17]:
model.save(final_model_path)
print(f"Final model saved to {final_model_path}")

Final model saved to model\final_model.h5


## Cargar un modelo creado previamente

In [12]:
def load_trained_model(model_path):
    # Find the most recent model directory
    
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found at {model_path}")
    
    # Load the model
    model = load_model(model_path)
    print(f"Model loaded successfully from {model_path}")
    return model


In [14]:
loaded_model = load_trained_model("model/final_model.h5")


Model loaded successfully from model/final_model.h5
